# TP2
## Grupo 17:
**PG50315 - David Alexandre Ferreira Duarte**

**PG51247 - João Rafael Cerqueira Monteiro**

## Exercício 2.
    
Construir uma classe Python que implemente o  EdCDSA a partir do “standard” FIPS186-5

    A. A implementação deve conter funções para assinar digitalmente e verificar a assinatura.
    B. A implementação da classe deve usar  uma das “Twisted Edwards Curves” definidas no standard e escolhida  na iniciação da classe: a curva  “edwards25519” ou “edwards448”.
    C. Por aplicação da transformação de Fiat-Shamir construa um protocolo de autenticação de desafio-resposta.

In [55]:
import hashlib
import secrets

In [56]:
class TwistedEdwardsCurve(object):
    def __init__(self, ed = None):
        if ed == 'ed25519':
            self.p = 2^255-19
            self.K = GF(self.p)
            self.a = self.K(-1)
            self.d = -self.K(121665)/self.K(121666)
            self.ed25519 = {
                'b'  : 256,
                'Px' : self.K(15112221349535400772501151409588531511454012693041857206046113283949847762202),
                'Py' : self.K(46316835694926478169428394003475163141307993866256225615783033603165251855960),
                'L'  : ZZ(2^252 + 27742317777372353535851937790883648493), ## ordem do subgrupo primo
                'n'  : 254,
                'h'  : 8
            }
        else:
            # Edwards 448
            self.p = 2^448 - 2^224 - 1
            self.K = GF(self.p)
            self.a = self.K(1)
            self.d = self.K(-39081)
            self.ed448= {
                'b'  : 456,     ## tamanho das assinaturas e das chaves públicas
                'Px' : self.K(224580040295924300187604334099896036246789641632564134246125461686950415467406032909029192869357953282578032075146446173674602635247710) ,
                'Py' : self.K(298819210078481492676017930443930673437544040154080242095928241372331506189835876003536878655418784733982303233503462500531545062832660) ,                                          
                'L'  : ZZ(2^446 - 13818066809895115352007386748515426880336692474882178609894547503885) ,
                'n'  : 447,     ## tamanho dos segredos: os dois primeiros bits são 0 e o último é 1.
                'h'  : 4        ## cofactor
            }
            
        assert self.a != self.d and is_prime(self.p) and self.p > 3
        K        = GF(self.p) 
  
        A =  2*(self.a + self.d)/(self.a - self.d)
        B =  4/(self.a - self.d)
    
        alfa = A/(3*B) ; s = B

        a4 =  s^(-2) - 3*alfa^2
        a6 =  -alfa^3 - a4*alfa
        
        self.K = K
        self.constants = {'a': self.a , 'd': self.d , 'A':A , 'B':B , 'alfa':alfa , 's':s , 'a4':a4 , 'a6':a6 }
        self.EC = EllipticCurve(K,[a4,a6]) 
        
        if ed == 'ed25519':
            self.L = self.ed25519['L']
            self.P = self.ed2ec(self.ed25519['Px'],self.ed25519['Py'])  # gerador do gru
        else:
            self.gen()
    
    def order(self):
        # A ordem prima "n" do maior subgrupo da curva, e o respetivo cofator "h" 
        oo = self.EC.order()
        n,_ = list(factor(oo))[-1]
        return (n,oo//n)
    
    def gen(self):
        L, h = self.order()       
        P = O = self.EC(0)
        while L*P == O:
            P = self.EC.random_element()
        self.P = h*P ; self.L = L

  
    
    def is_edwards(self, x, y):
        a = self.constants['a'] ; d = self.constants['d']
        x2 = x^2 ; y2 = y^2
        return a*x2 + y2 == 1 + d*x2*y2

    def ed2ec(self,x,y):      ## mapeia Ed --> EC
        if (x,y) == (0,1):
            return self.EC(0)
        z = (1+y)/(1-y) ; w = z/x
        alfa = self.constants['alfa']; s = self.constants['s']
        return self.EC(z/s + alfa , w/s)
    
    def ec2ed(self,P):        ## mapeia EC --> Ed
        if P == self.EC(0):
            return (0,1)
        x,y = P.xy()
        alfa = self.constants['alfa']; s = self.constants['s']
        u = s*(x - alfa) ; v = s*y
        return (u/v , (u-1)/(u+1))

In [57]:
class EdCDSA:
    def __init__(self, curve, private_key):
        self.curve = curve
        self.private_key = private_key
        self.public_key = self.private_key * self.curve.P

    def sign(self, message):
        # Gera um número aleatório k
        k = secrets.randbelow(self.curve.L)

        # Calcula o ponto R = k * P
        R = k * self.curve.P

        # Calcula o valor h da hash da mensagem
        h = int(hashlib.sha512(message).hexdigest(), 16)

        # Calcula o valor s = (h + x(R) * d) * k^(-1) mod L
        x_R = R.xy()[0]
        s = (h + x_R * self.private_key) * inverse_mod(k, self.curve.L) % self.curve.L

        # Retorna a assinatura (R, s)
        return (R, s)

    def verify(self, message, signature):
        R, s = signature

        # Verifica se o ponto R está na curva torcida de Edwards
        if not self.curve.EC(R):
            return False

        # Calcula o valor v = H(m) * B / s - r * B / s
        hash_message = int.from_bytes(hashlib.sha256(message.encode()).digest(), "big")
        r = R.xy()[0]
        v = (hash_message * self.curve.P + (-r) * self.public_key) * pow(s, -1, self.curve.L)

        # Verifica se v * P = R - H(m) * A / s
        if v * self.curve.P == R - hash_message * self.public_key * pow(s, -1, self.curve.L):
            return True
        else:
            return False

    @staticmethod
    def fiat_shamir(challenge, secret):
        # Transformação de Fiat-Shamir
        hash_challenge = int.from_bytes(hashlib.sha256(challenge).digest(), "big")
        return EdCDSA(TwistedEdwardsCurve('ed25519'), secret).sign(hash_challenge.to_bytes((hash_challenge.bit_length() + 7) // 8, "big"))

In [58]:
# Exemplo de uso
curve = TwistedEdwardsCurve('ed25519')
print(curve.order())

(7237005577332262213973186563042994240857116359379907606001950938285454250989, 8)


In [59]:
private_key = secrets.randbelow(curve.order()[0])
edcdsa = EdCDSA(curve, private_key)

# Protocolo de autenticação de desafio-resposta usando a transformação de Fiat-Shamir
# Signatário envia sua chave pública
public_key = edcdsa.public_key

# Verificador envia um desafio aleatório
challenge = secrets.token_bytes(32)

# Signatário calcula a resposta assinando o desafio usando o EdCDSA e envia a resposta para o verificador
response = EdCDSA.fiat_shamir(challenge, private_key)

# Verificador verifica a resposta usando a chave pública do signatário e o protocolo de verificação de assinatura do EdCDSA
if edcdsa.verify(challenge, response):
    print("Autenticado com sucesso!")
else:
    print("Falha na autenticação.")

ArithmeticError: reduction modulo 7237005577332262213973186563042994240857116359379907606001950938285454250989 not defined